In [ ]:
!pip install ultralytics

## Đối với Object Detection 

In [ ]:
from ultralytics import YOLO  
import os 

# Vô hiệu hóa tính năng WandB để không tự động gửi dữ liệu lên WandB trong quá trình huấn luyện
os.environ['WANDB_DISABLED'] = 'True'

# Tải mô hình YOLOv8 với trọng số đã được huấn luyện trước (YOLOv8m)
model = YOLO(model='yolov8m.pt')  

# Huấn luyện mô hình với bộ dữ liệu tùy chỉnh
results = model.train(
    data="/your/path/yaml-1/wildtrack.yaml",  # Đường dẫn đến tệp dữ liệu YAML
    epochs=100,  # Số lượng epochs để huấn luyện mô hình
    imgsz=640,  # Kích thước hình ảnh đầu vào (640x640 pixels)
    project="yolov8_training",  # Tên thư mục chứa kết quả huấn luyện
    name="results"  # Tên của thư mục con trong "yolov8_training" để lưu trữ kết quả
)

# Di chuyển các trọng số đã lưu sau khi huấn luyện đến thư mục mong muốn
import shutil  # Nhập thư viện shutil để di chuyển file
shutil.move("yolov8_training/results", "/your/path/yolov8_training")  # Di chuyển thư mục kết quả đến đường dẫn mới


## Đối với Object Tracking

### Tracking qua từng frame

In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv8
model = YOLO("/your/path/weight-last-m/best_last_m.pt")

# Thực hiện tracking trên frame
results = model.track(source="/kaggle/input/camera-1/C1", conf=0.3, iou=0.5, persist=True, stream=True)

# Mở file TXT để ghi kết quả
with open('tracking_results_new.txt', mode='w') as file:
    # Vòng lặp qua các kết quả tracking
    for frame_idx, result in enumerate(results):
        boxes = result.boxes.xyxy  # Bounding boxes (x1, y1, x2, y2)
        ids = result.boxes.id      # Object IDs
        confs = result.boxes.conf  # Confidence scores

        for i in range(len(boxes)):
            # Chuyển đổi từ (x1, y1, x2, y2) sang (bb_left, bb_top, bb_width, bb_height)
            bb_left = boxes[i][0]    # x1
            bb_top = boxes[i][1]     # y1
            bb_width = boxes[i][2] - boxes[i][0]  # x2 - x1
            bb_height = boxes[i][3] - boxes[i][1]  # y2 - y1

            # Ghi kết quả theo định dạng: frame, object_id, bb_left, bb_top, bb_width, bb_height, confidence
            file.write(f"{frame_idx}, {ids[i]}, "
                       f"{bb_left}, {bb_top}, {bb_width}, {bb_height}, {confs[i]:.2f}\n")


### Tracking trên 1 video

In [ ]:
import cv2

from ultralytics import YOLO

# Tải mô hình YOLOv8
model = YOLO("/your/path/weight-last-m/best_last_m.pt")

# Mở tệp video
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(video_path)

# Vòng lặp để xử lý từng khung hình của video
while cap.isOpened():
    # Đọc một khung hình từ video
    success, frame = cap.read()

    if success:
        # Chạy theo dõi đối tượng bằng YOLOv8 trên khung hình, duy trì các đối tượng giữa các khung hình
        results = model.track(frame, persist=True)

        # Vẽ kết quả nhận dạng đối tượng lên khung hình
        annotated_frame = results[0].plot()

        # Hiển thị khung hình đã được chú thích
        cv2.imshow("Theo dõi bằng YOLOv8", annotated_frame)

        # Thoát vòng lặp nếu nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Thoát vòng lặp nếu đến cuối video
        break

# Giải phóng tài nguyên và đóng cửa sổ hiển thị
cap.release()
cv2.destroyAllWindows()
